# Building Data Genome 2.0
## All-meters datasets creation: hourly and daily timestamp

Biam! (pic.biam@gmail.com)

In [2]:
import pandas as pd
import numpy as np
from glob import glob

# Meters data

To join all meters raw dataets run the following code.

## Data join

In [3]:
# Write custom path here
path_raw = "..\\data\\meters\\raw\\"
path_proc = "..\\data\\meters\\processed\\"

In [4]:
# files in directory
files = glob(path_raw + "*.csv")

In [5]:
files

['..\\data\\meters\\raw\\chilledwater.csv',
 '..\\data\\meters\\raw\\electricity.csv',
 '..\\data\\meters\\raw\\gas.csv',
 '..\\data\\meters\\raw\\hotwater.csv',
 '..\\data\\meters\\raw\\irrigation.csv',
 '..\\data\\meters\\raw\\solar.csv',
 '..\\data\\meters\\raw\\steam.csv',
 '..\\data\\meters\\raw\\water.csv']

In [6]:
dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("\\")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
complete_data = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

In [9]:
complete_data.tail()

,timestamp,building_id,meter_reading,meter
53561827,2017-12-31 19:00:00,Wolf_education_Ursula,55.4761,water
53561828,2017-12-31 20:00:00,Wolf_education_Ursula,50.1927,water
53561829,2017-12-31 21:00:00,Wolf_education_Ursula,52.8344,water
53561830,2017-12-31 22:00:00,Wolf_education_Ursula,55.4761,water
53561831,2017-12-31 23:00:00,Wolf_education_Ursula,55.4761,water


In [7]:
# Check the meters
complete_data["meter"].value_counts()

electricity     27684432
chilledwater     9736920
steam            6491280
hotwater         3245640
gas              3105288
water            2561424
irrigation        649128
solar              87720
Name: meter, dtype: int64

In [8]:
# Check number of buildings
len(np.unique(complete_data["building_id"]))

1636

In [9]:
# Rounds to 4 decimals
complete_data["meter_reading"] = round(complete_data["meter_reading"],4)

## Hourly timestamp

Meters reading are currently hourly.

In [10]:
complete_data.to_csv(path_proc + "allmeters.csv", index=False)

## Daily timestamp

Daily readings are summarized as the sum of all the readings of that day.

In [11]:
# Converts timestamp to datetime object
complete_data["timestamp"] = pd.to_datetime(complete_data["timestamp"], format='%Y-%m-%d %H:%M:%S')

In [12]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53561832 entries, 0 to 53561831
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp      datetime64[ns]
 1   building_id    object        
 2   meter_reading  float64       
 3   meter          object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.6+ GB


In [13]:
# Sums all readings per day
complete_data = complete_data.groupby(by=["building_id","meter",complete_data['timestamp'].dt.date]).sum()

In [14]:
# Reset indexes to ungroup
complete_data = complete_data.reset_index()

In [15]:
complete_data.head()

,building_id,meter,timestamp,meter_reading
0,Bear_assembly_Angel,electricity,2016-01-01,12808.1620
1,Bear_assembly_Angel,electricity,2016-01-02,9251.0003
2,Bear_assembly_Angel,electricity,2016-01-03,14071.6500
3,Bear_assembly_Angel,electricity,2016-01-04,12860.3758
4,Bear_assembly_Angel,electricity,2016-01-05,12212.8585


In [16]:
# export csv
complete_data.to_csv(path_proc + "allmeters_daily.csv", index=False)

In [ ]:
del complete_data